In [2]:
%load_ext autoreload
%autoreload 2

import torch 
import os
import glob
from matplotlib import pyplot as plt

import matplotlib.animation as animation
import os
import torch
import numpy as np
import pandas as pd
# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')


from distributions import ZeroInflatedDist, QuantizedNormal
from torch_models import MixtureOfTruncNormModel, SpatialWaves
from torch_training import train_epoch
from torch_perturb.torch_pert_topk import PerturbedBrokenTopK
from torch_perturb.perturbations import perturbed
import time
from torch_training import train_epoch_largesynth
from torch_models import torch_bpr_uncurried, deterministic_bpr

In [3]:
seed=360
num_components=2
init_idx=10
perturbed_noise=0.05
step_size=0.1
threshold = 1
bpr_weight=30
nll_weight=0

In [4]:
# total timepoints
T= 500
S=7
K=5

dist_S = [QuantizedNormal(10, 0.3),
        QuantizedNormal(20, 0.3),
        QuantizedNormal(30, 0.3),
        QuantizedNormal(40, 0.3),
        QuantizedNormal(50, 0.3),
        QuantizedNormal(60, 0.3),
        QuantizedNormal(100, 0.3)]
train_y_TS = np.zeros((T, S))
for s, dist in enumerate(dist_S):
    random_state = np.random.RandomState(10000 * seed + s*123456)
    train_y_TS[:, s] = dist.rvs(size=T, random_state=random_state)
model = MixtureOfTruncNormModel(num_components=num_components, S=S, low=0, high=150)

In [5]:
means = [20,70]
scales = [5, 5]
softinv_means = torch.tensor(means) + torch.log(-torch.expm1(-torch.tensor(means)))
softinv_scales = torch.tensor(scales) - 0.2 + torch.log1p(-torch.expm1(-torch.tensor(scales)+0.2))
all_mix_weights = np.random.dirichlet([0.5]*2, (20,S))
mix_weights = torch.log(1e-13 + torch.tensor(all_mix_weights[10]))
#mix_weights = torch.log(1e-13 + torch.tensor(np.array([[1,0], [1,0], [1,0], [1,0], [1,0], [0,1], [0,1]])).view(-1))
model.update_params(torch.cat([softinv_means, softinv_scales, mix_weights.view(-1)]))

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=step_size)

M_score_func =  100
M_action = 100
train_T = train_y_TS.shape[0]
perturbed_top_K_func = PerturbedBrokenTopK(k=K, sigma=perturbed_noise)
losses, bprs, nlls = [], [], []
for epoch in range(100):
    print(f'EPOCH: {epoch}')
    loss, bpr, nll, model = train_epoch(model, optimizer, K, threshold, train_T, M_score_func, M_action, train_y_TS, perturbed_top_K_func, bpr_weight, nll_weight)
    losses.append(loss)
    bprs.append(bpr)
    nlls.append(nll)

EPOCH: 0
det bpr: 0.5741160172746886
Pert bpr: 0.6978899587713763
nll: 10.779538792350333
Loss: -20.936698763141287
EPOCH: 1
det bpr: 0.5888968418296666
Pert bpr: 0.7085127566233654
nll: 10.56899701131243
Loss: -21.255382698700963
EPOCH: 2
det bpr: 0.6460909949124694
Pert bpr: 0.7194063760584265
nll: 10.483844134077831
Loss: -21.582191281752795
EPOCH: 3
det bpr: 0.7425484622060646
Pert bpr: 0.7279246558251284
nll: 10.434613011084178
Loss: -21.837739674753852
EPOCH: 4
det bpr: 0.822193971993744
Pert bpr: 0.7362336255315256
nll: 10.401852462917923
Loss: -22.087008765945768
EPOCH: 5
det bpr: 0.8826359785695461
Pert bpr: 0.7471752495477062
nll: 10.378988928627642
Loss: -22.415257486431187
EPOCH: 6
det bpr: 0.8897140689161059
Pert bpr: 0.757066136597078
nll: 10.378496872171949
Loss: -22.71198409791234
EPOCH: 7
det bpr: 0.8929260668825523
Pert bpr: 0.7664924833126118
nll: 10.391113743287
Loss: -22.99477449937835
EPOCH: 8
det bpr: 0.8929260668825523
Pert bpr: 0.776812018898788
nll: 10.4086075